In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time
import random
import os
from dotenv import load_dotenv, dotenv_values

In [2]:
team = 'ARI'
year = '2023'
URL = f'https://www.footballguys.com/stats/game-logs-against/teams?team={team}&year={year}'

In [3]:
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'lxml')

In [4]:
teams_raw = soup.find('select',{'class':'big-select'})
teams_options = teams_raw.find_all('option')
teams = []
for team in teams_options:
    teams.append(team['value'])

In [5]:
tables = soup.find_all('table',{'class':'table sortable-table'})
df = pd.read_html(StringIO(str(tables[0])))[0]
df = df[0:0]

In [6]:
def table_parser(df, team, year):
    URL = f'https://www.footballguys.com/stats/game-logs-against/teams?team={team}&year={year}'
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'lxml')
    tables = soup.find_all('table',{'class':'table sortable-table'})
    temp_df = pd.read_html(StringIO(str(tables[0])))[0]
    temp_df['pos'] = 'qb'
    temp_df['opp'] = team
    df = pd.concat([df, temp_df])
    temp_df = pd.read_html(StringIO(str(tables[1])))[0]
    temp_df['pos'] = 'rb'
    temp_df['opp'] = team
    df = pd.concat([df, temp_df])
    temp_df = pd.read_html(StringIO(str(tables[2])))[0]
    temp_df['pos'] = 'wr'
    temp_df['opp'] = team
    df = pd.concat([df, temp_df])
    temp_df = pd.read_html(StringIO(str(tables[3])))[0]
    temp_df['pos'] = 'te'
    temp_df['opp'] = team
    df = pd.concat([df, temp_df])
    return df

In [7]:
for team in teams:
    df = table_parser(df, team, year)
    time.sleep(random.uniform(2,5))

In [8]:
df['team-pos'] = df['team'] + '-' + df['pos']
df['name-pos-team'] = df['name'] + '-' + df['pos'] + '-' + df['team']

In [9]:
df_sum_teampos = df.copy().drop(columns=['name'])
df_sum_teampos = df_sum_teampos.groupby(by=['name-pos-team', 'team', 'pos', 'week', 'opp', 'team-pos']).sum()
df_sum_teampos = df_sum_teampos.reset_index()
df_sum_teampos.head(5)

,name-pos-team,team,pos,week,opp,team-pos,cmp,att,pyd,ptd,int,rsh,rshyd,rshtd,targ,rec,recyd,rectd
0,A.J. Brown-wr-PHI,PHI,wr,1,NE,PHI-wr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,7.0,79.0,0.0
1,A.J. Brown-wr-PHI,PHI,wr,2,MIN,PHI-wr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,29.0,0.0
2,A.J. Brown-wr-PHI,PHI,wr,3,TB,PHI-wr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,9.0,131.0,0.0
3,A.J. Brown-wr-PHI,PHI,wr,4,WAS,PHI-wr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,9.0,175.0,2.0
4,A.J. Brown-wr-PHI,PHI,wr,5,LAR,PHI-wr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,6.0,127.0,0.0


In [10]:
df_avg_teampos = df_sum_teampos.copy().drop(columns=['opp', 'week'])
df_avg_teampos = df_avg_teampos.groupby(by=['name-pos-team', 'team', 'pos', 'team-pos']).mean()
df_avg_teampos = df_avg_teampos.reset_index()
df_avg_teampos.head(5)

,name-pos-team,team,pos,team-pos,cmp,att,pyd,ptd,int,rsh,rshyd,rshtd,targ,rec,recyd,rectd
0,A.J. Brown-wr-PHI,PHI,wr,PHI-wr,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9.600000,6.800000,101.300000,0.600000
1,A.T. Perry-wr-NO,NO,wr,NO-wr,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,4.000000,2.000000,38.000000,1.000000
2,AJ Dillon-rb-GB,GB,rb,GB-rb,0.0,0.0,0.0,0.0,0.0,11.700000,40.5,0.100000,2.100000,1.600000,14.600000,0.000000
3,Aaron Jones-rb-GB,GB,rb,GB-rb,0.0,0.0,0.0,0.0,0.0,9.428571,35.0,0.285714,4.285714,2.714286,24.142857,0.142857
4,Adam Prentice-rb-NO,NO,rb,NO-rb,0.0,0.0,0.0,0.0,0.0,1.000000,5.0,0.000000,2.000000,1.000000,5.000000,0.000000


In [11]:
df_strength_merge = pd.merge(df_sum_teampos, df_avg_teampos, on=['name-pos-team'], how='left')
df_strength_merge.head()

,name-pos-team,team_x,pos_x,week,opp,team-pos_x,cmp_x,att_x,pyd_x,ptd_x,...,pyd_y,ptd_y,int_y,rsh_y,rshyd_y,rshtd_y,targ_y,rec_y,recyd_y,rectd_y
0,A.J. Brown-wr-PHI,PHI,wr,1,NE,PHI-wr,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.6,6.8,101.3,0.6
1,A.J. Brown-wr-PHI,PHI,wr,2,MIN,PHI-wr,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.6,6.8,101.3,0.6
2,A.J. Brown-wr-PHI,PHI,wr,3,TB,PHI-wr,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.6,6.8,101.3,0.6
3,A.J. Brown-wr-PHI,PHI,wr,4,WAS,PHI-wr,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.6,6.8,101.3,0.6
4,A.J. Brown-wr-PHI,PHI,wr,5,LAR,PHI-wr,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.6,6.8,101.3,0.6


In [12]:
df_strength_raw = pd.DataFrame()
df_strength_raw['name-pos-team'] = df_strength_merge['name-pos-team']
df_strength_raw['opp'] = df_strength_merge['opp']
df_strength_raw['pos'] = df_strength_merge['pos_x']
df_strength_raw['team'] = df_strength_merge['team_x']
df_strength_raw['week'] = df_strength_merge['week']
df_strength_raw['att'] = df_strength_merge['att_x'] - df_strength_merge['att_y']
df_strength_raw['cmp'] = df_strength_merge['cmp_x'] - df_strength_merge['cmp_y']
df_strength_raw['pyd'] = df_strength_merge['pyd_x'] - df_strength_merge['pyd_y']
df_strength_raw['ptd'] = df_strength_merge['ptd_x'] - df_strength_merge['ptd_y']
df_strength_raw['int'] = df_strength_merge['int_x'] - df_strength_merge['int_y']
df_strength_raw['rsh'] = df_strength_merge['rsh_x'] - df_strength_merge['rsh_y']
df_strength_raw['rshyd'] = df_strength_merge['rshyd_x'] - df_strength_merge['rshyd_y']
df_strength_raw['rshtd'] = df_strength_merge['rshtd_x'] - df_strength_merge['rshtd_y']
df_strength_raw['targ'] = df_strength_merge['targ_x'] - df_strength_merge['targ_y']
df_strength_raw['rec'] = df_strength_merge['rec_x'] - df_strength_merge['rec_y']
df_strength_raw['recyd'] = df_strength_merge['recyd_x'] - df_strength_merge['recyd_y']
df_strength_raw['rectd'] = df_strength_merge['rectd_x'] - df_strength_merge['rectd_y']
df_strength_raw.head()

,name-pos-team,opp,pos,team,week,att,cmp,pyd,ptd,int,rsh,rshyd,rshtd,targ,rec,recyd,rectd
0,A.J. Brown-wr-PHI,NE,wr,PHI,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.2,-22.3,-0.6
1,A.J. Brown-wr-PHI,MIN,wr,PHI,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.6,-2.8,-72.3,-0.6
2,A.J. Brown-wr-PHI,TB,wr,PHI,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.4,2.2,29.7,-0.6
3,A.J. Brown-wr-PHI,WAS,wr,PHI,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4,2.2,73.7,1.4
4,A.J. Brown-wr-PHI,LAR,wr,PHI,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.6,-0.8,25.7,-0.6


In [13]:
df_strength_sum = df_strength_raw.copy().drop(columns=['name-pos-team'])
df_strength_sum = df_strength_sum.groupby(by=['opp', 'pos', 'team', 'week']).sum()
df_strength_sum = df_strength_sum.reset_index()
df_strength_sum.head(5)

,opp,pos,team,week,att,cmp,pyd,ptd,int,rsh,rshyd,rshtd,targ,rec,recyd,rectd
0,ARI,qb,ATL,10,-31.000000,-19.555556,-265.333333,-0.666667,-1.000000,0.444444,5.666667,0.555556,0.0,0.0,0.0,0.0
1,ARI,qb,BAL,8,-0.454545,-1.090909,-64.909091,-0.090909,-0.454545,-4.181818,-31.636364,-0.454545,0.0,0.0,0.0,0.0
2,ARI,qb,CIN,5,9.500000,11.600000,86.100000,1.500000,0.400000,0.900000,-1.800000,0.000000,0.0,0.0,0.0,0.0
3,ARI,qb,CLE,9,1.500000,1.500000,33.166667,0.833333,-0.666667,-1.333333,-1.666667,-0.166667,0.0,0.0,0.0,0.0
4,ARI,qb,DAL,3,6.200000,1.300000,-11.400000,-0.900000,0.400000,-0.300000,9.900000,-0.200000,0.0,0.0,0.0,0.0


In [14]:
df_strength = df_strength_sum.copy().drop(columns=['team', 'week'])
df_strength = df_strength.groupby(by=['opp', 'pos']).mean()
df_strength = df_strength.reset_index()
df_strength['fpts'] = (df_strength['pyd'] * 0.04) + (df_strength['ptd'] * 5) + (df_strength['int'] * -2) + (df_strength['rshyd'] * 0.1) + (df_strength['rshtd'] * 6) + (df_strength['rec'] * 0.5) + (df_strength['recyd'] * 0.1) + (df_strength['rectd'] * 6)
df_strength.head()

,opp,pos,att,cmp,pyd,ptd,int,rsh,rshyd,rshtd,targ,rec,recyd,rectd,fpts
0,ARI,qb,-3.494215,-0.927089,-10.733517,0.182369,-0.000092,-0.290450,-2.366942,0.180533,0.000000,0.000000,0.000000,0.000000,1.329190
1,ARI,rb,0.000000,0.000000,0.000000,0.000000,0.000000,5.624567,23.461442,0.288872,0.283068,0.710842,-0.469667,0.120661,5.111800
2,ARI,te,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.188571,-1.000397,-12.814008,-0.019722,-1.899933
3,ARI,wr,0.000000,0.000000,0.000000,0.000000,0.000000,-0.191437,-0.999197,-0.060537,-0.024190,1.160354,24.800112,0.115342,3.289096
4,ATL,qb,-3.492626,-2.696818,-36.806515,0.353030,-0.297980,0.479798,3.308333,0.068485,0.000000,0.000000,0.000000,0.000000,1.630593


In [15]:
load_dotenv()
df_strength.to_csv(f'{os.getenv('FILES_DIR')}/df_strength_2.csv')